In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

from federated_learning.attacks.MemberInference.metrics import *
from federated_learning.attacks.MemberInference.train import *

from federated_learning.model import LeNet_Small_Quant

import federated_learning

device = torch.device("cuda" if torch.cuda.is_available() else "mps")

In [3]:
n_epochs = 10
batch_size = 32
lr = 0.001

In [4]:
(X_train, y_train), (X_test, y_test) = federated_learning.load_cifar10(num_users=1, n_class=10, n_samples=1000, even_split=True)
X_train, y_train = X_train[0], y_train[0]

X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()

train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

Files already downloaded and verified
Files already downloaded and verified


In [5]:
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


def imshow(img):
    img /= 255
    plt.imshow(img)
    plt.show()

In [6]:
total_size = len(X_train)

indices = list(range(total_size))   

split = total_size // 2
target_train_idx = indices[:split]
target_out_idx = indices[split:]

target_train_sampler = SubsetRandomSampler(target_train_idx)
target_out_sampler = SubsetRandomSampler(target_out_idx)

target_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=target_train_sampler, num_workers=1)
target_out_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=target_out_sampler, num_workers=1)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

target_net = LeNet_Small_Quant().to(device) 
target_loss = nn.CrossEntropyLoss()
target_optim = optim.Adam(target_net.parameters(), lr=lr)


In [7]:
train(target_net, target_train_loader, test_loader, target_optim, target_loss, n_epochs, classes=classes)

[0/10]
Training:
Accuracy of airplane: 39.731285988483684
Accuracy of automobile: 57.947686116700204
Accuracy of bird: 44.72049689440994
Accuracy of cat: 41.90871369294606
Accuracy of deer: 0.20202020202020202
Accuracy of dog: 1.3725490196078431
Accuracy of frog: 56.39412997903564
Accuracy of horse: 16.274509803921568
Accuracy of ship: 23.25581395348837
Accuracy of truck: 41.45383104125737
Accuracy: 32.08
Test:
Accuracy of airplane: 43.1
Accuracy of automobile: 57.3
Accuracy of bird: 43.1
Accuracy of cat: 37.0
Accuracy of deer: 0.6
Accuracy of dog: 0.7
Accuracy of frog: 46.5
Accuracy of horse: 16.9
Accuracy of ship: 18.9
Accuracy of truck: 39.5
Accuracy: 30.36
[1/10]
Training:
Accuracy of airplane: 46.83301343570058
Accuracy of automobile: 7.645875251509055
Accuracy of bird: 12.422360248447205
Accuracy of cat: 46.47302904564315
Accuracy of deer: 20.2020202020202
Accuracy of dog: 25.098039215686274
Accuracy of frog: 47.16981132075472
Accuracy of horse: 75.09803921568627
Accuracy of ship

(83.26, 40.17)

In [9]:
from mia.estimators import ShadowModelBundle

Shadow_dataset_size = 2000
num_shadows = 2

In [10]:
smb = ShadowModelBundle(target_net,
                        shadow_dataset_size=Shadow_dataset_size,
                        num_models=num_shadows)

In [13]:
from mia.wrappers import TorchWrapper

model = TorchWrapper(LeNet_Small_Quant(), nn.CrossEntropyLoss, torch.optim.Adam)

TypeError: LeNet_Small_Quant.forward() missing 1 required positional argument: 'x'